In [70]:
# Kaggle入门竞赛，数字识别，使用pytorch实现LeNet

# pytorch 版本 0.4.0

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
from torch.autograd import Variable

In [47]:
# 数字识别的图片大小为 1 * 28 * 28

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        # 1 * 28 * 28
        x = F.relu(self.conv1(x))
        # 6 * 24 * 24
        x = self.pool(x)
        # 6 * 12 * 12
        x = F.relu(self.conv2(x))
        # 16 * 8 * 8
        x = self.pool(x)
        # 16 * 4 * 4
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        # 120
        x = F.relu(self.fc2(x))
        # 84
        x = self.fc3(x)
        # 10
        return x
    
net = LeNet()
print(net)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [48]:
IS_USE_GPU = torch.cuda.is_available()
if IS_USE_GPU:
    net = net.cuda()
    print ('USE GPU')
else:
    print ('USE CPU')

USE GPU


In [49]:
train_address = "train.csv"
test_address = "test.csv"


train = pd.read_csv(train_address).values
test  = pd.read_csv(test_address).values

train_data = train[:, 1:].reshape(train.shape[0], 1, 28, 28)
train_data = train_data.astype(float)
train_data = train_data / 255
train_data = torch.from_numpy(train_data)

train_label = train[:, 0]
train_label = train_label.astype(int)
train_label = torch.from_numpy(train_label)
# 此时的label是一维的，我们把它变成二维只有一列的数组
train_label = train_label.view(train.shape[0], -1)

test_data = test.reshape(test.shape[0], 1, 28, 28)
test_data = test_data.astype(float)
test_data = test_data / 255
test_data = torch.from_numpy(test_data)

print ("Loading data finished...")

Loading data finished...


In [50]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum = 0.9)

In [77]:
# training model
epoch_C = 5
batch = 5
# 让GPU加速更快，batch就需要加大
# 但是batch增大会使网络收敛增大变大

for epoch in range(epoch_C):
    loss = 0.0
    index = 0
    while (index + batch) < train_data.shape[0]:
        loss = 0.0
        data = Variable(train_data[index : index+batch].clone())
        label = Variable(train_label[index : index+batch].clone())
        label = label.view(batch)
        
        data  = data.type(torch.FloatTensor)
        label = label.type(torch.LongTensor)
        
        if IS_USE_GPU:
            data  = data.cuda()
            label = label.cuda()
            
        optimizer.zero_grad()
        out = net(data)
        loss = criterion(out, label)
        loss.backward()
        optimizer.step()
        # if ((index) % 2000 == 0):
            # print("Epoch = %d,Index = %d,  Loss = %f" %(epoch, index, loss))
        index = index + batch
        
    print("Epoch = %d,  Loss = %f" %(epoch, loss))
print("training finished")

Epoch = 0,  Loss = 0.000000
Epoch = 1,  Loss = 0.000000
Epoch = 2,  Loss = 0.000000
Epoch = 3,  Loss = 0.000000
Epoch = 4,  Loss = 0.000000
training finished


In [74]:
# test
count = test_data.shape[0]
final_prediction = np.ndarray(shape = (count, 2), dtype=int)
for index in range(count):
    data = Variable(test_data[index : index+1].clone())
    data = data.type(torch.FloatTensor)
    if IS_USE_GPU:
        data = data.cuda()
    out = net(data)
    pred = torch.max(out, 1)
    final_prediction[index][0] = 1 + index
    final_prediction[index][1] = pred[1][0]
    if (index + 1) % 2000 == 0:
        print("Total tested = %d" %(index + 1))

Total tested = 2000
Total tested = 4000
Total tested = 6000
Total tested = 8000
Total tested = 10000
Total tested = 12000
Total tested = 14000
Total tested = 16000
Total tested = 18000
Total tested = 20000
Total tested = 22000
Total tested = 24000
Total tested = 26000
Total tested = 28000


In [75]:
submission = pd.DataFrame(final_prediction, dtype=int, columns=['ImageId', 'Label'])
submission.to_csv('submission.csv', index=False, header=True)